$$\frac{dms(t)}{dt} = csf*Fin - qs *cx*Vl$$

$$\frac{dmx(t)}{dt} =  µ *cx*Vl$$

$$Fin = \frac{dVl(t)}{dt}$$

In [1]:
import numpy as np
from scipy.integrate import odeint, solve_ivp
import matplotlib.pyplot as plt
import pandas as pd
import cufflinks as cf
import chart_studio

import plotly as py
from plotly.offline import iplot
#chart_studio.tools.set_credentials_file(username='name', api_key='key') #von chart_studio auf plotly page


# Enter qsmax in g/g*h
# Enter Ks in g/l
# Enter µmax in 1/h
# Enter entrance substrate concentration
# enter inlet flow in l/h
# enter the end vloume of the reactor

qsmax = 0.2
Ks = 2
mumax = 0.5 
csf = 50
Fin = 2
Vend = 20



# def model (y, t):  # for use with old odeint
def model (t, y):  # for use with new solve_ivp
    
    ms = y[0]
    
    mx = y[1]
    
    Vl = y[2]
    
    
    cs = ms/Vl
    
    cx = mx/Vl
    
    qs = qsmax* cs/(cs + Ks)

    mu = mumax*cs/(cs + Ks)
    
   
    if Vl <= Vend:
        dVldt = Fin
    else:
        dVldt = 0.0
        
    
    dmsdt = csf*dVldt - qs *cx*Vl
    
    #Zulauf ohne Biomasse
    dmxdt = mu*cx*Vl
    
    return (dmsdt, dmxdt, dVldt)


t = np.linspace(0, 15, 100)

y0 = [250, 150, 5]

#result = odeint(model, y0, t) # old: odeint
result = solve_ivp(model, [np.min(t), np.max(t)], y0, t_eval=t).y.T # new: solve_ivp




pd.set_option('display.max_rows', 10)
pandaframe = pd.DataFrame(result, columns= ("ms", "mx", "Vl"), index = t) #,index = np.round(t*60, 2) for min
pandaframe.index.name = ("time in h")

print(pandaframe)  # wenn man das nach dem plot printed kommt kein plot mehr

layout = dict(title = "Substratemass,Biomass, Volume over time", xaxis= dict(title= "time in h"),  yaxis = dict(title= "mass in g and volume in l"))

# in yaxis: type = "log", # yaxis: rangemode: 'tozero'# oder range=[fromValue,toValue]  wenn man log skala setzt sieht man das schwanken für ms vom solver im negativen bereich
#deshalb wäre es vielleicht besser die y skala erst bei 0 starten zu lasssen.
#oder die Schwankungen irgendwie zu glätten.

pandaframe.iplot(layout = layout )



                     ms           mx         Vl
time in h                                      
0.000000   2.500000e+02   150.000000   5.000000
0.151515   2.606193e+02   161.330477   5.303030
0.303030   2.708995e+02   173.508924   5.606061
0.454545   2.808156e+02   186.597436   5.909091
0.606061   2.903409e+02   200.662848   6.212121
...                 ...          ...        ...
14.393939  1.219117e-06  2651.630789  20.013046
14.545455 -4.907650e-07  2651.630793  20.013046
14.696970  6.276200e-07  2651.630790  20.013046
14.848485  2.895467e-07  2651.630791  20.013046
15.000000  1.936860e-07  2651.630791  20.013046

[100 rows x 3 columns]


In the following, the same with concentrations. And at the end multiplying the results with the respective volume to see if i get similar results like in the approach above.

$$\frac{dcs(t)}{dt} = \frac{Fin}{Vl}*(csf-cs) - qs *cx$$

$$\frac{dcx(t)}{dt} = cx * \left( µ-\frac{Fin}{Vl} \right)  $$





In [2]:
# Enter qsmax in g/g*h
# Enter Ks in g/l
# Enter µmax in 1/h
# Enter entrance substrate concentration
# enter inlet flow in l/h
# enter the end vloume of the reactor

qsmax = 0.2
Ks = 2
mumax = 0.5 
csf = 50
Fin = 2
Vend = 20


def model (t, y):  
    
    cs = y[0]
    
    cx = y[1]
    
    Vl = y[2]
    
    qs = qsmax* cs/(cs + Ks)

    mu = mumax*cs/(cs + Ks) 
    
    if Vl <= Vend:
        dVldt = Fin
    else:
        dVldt = 0.0
    
    dcsdt = dVldt/Vl * (csf - cs) - qs*cx
    
    dcxdt = cx*(mu - dVldt/Vl)
    
    return (dcsdt, dcxdt, dVldt)


t = np.linspace(0, 15, 100)

y0 = [50, 30, 5]

#result = odeint(model, y0, t) # old: odeint
result = solve_ivp(model, [np.min(t), np.max(t)], y0, t_eval=t).y.T 





pd.set_option('display.max_rows', 10)
pandaframe2 = pd.DataFrame(result, columns= ("cs", "cx", "Vl"), index = t)

pandaframe2.index.name = ("time in h")

print(pandaframe2)

layout2 = dict(title = "Substrateconc,Biomassconc, Volume over time", xaxis= dict(title= "time in h"),  yaxis = dict(  title= "conc in g/l and volume in l"))

pandaframe2.iplot(layout = layout2 )

#print(pandaframe2), printed my hier kommt kein plot mehr






                     cs          cx         Vl
time in h                                     
0.000000   5.000000e+01   30.000000   5.000000
0.151515   4.914535e+01   30.422303   5.303030
0.303030   4.832220e+01   30.949659   5.606061
0.454545   4.752189e+01   31.576949   5.909091
0.606061   4.673712e+01   32.300795   6.212121
...                 ...         ...        ...
14.393939 -4.524217e-07  132.499087  20.002439
14.545455  1.908758e-07  132.499085  20.002439
14.696970  1.043707e-08  132.499085  20.002439
14.848485  1.833079e-07  132.499085  20.002439
15.000000  6.173645e-07  132.499084  20.002439

[100 rows x 3 columns]


In [3]:
#Zur Kontrolle für die obere Zelle hier einfach die Ergebnisse der Konzentrationen mal das Volumen = selbes Ergebnis

cs_t = result[:, 0]
cx_t = result[:, 1]
Vl_t = result[:, 2]

ms = cs_t*Vl_t

mx = cx_t*Vl_t



pd.set_option('display.max_rows', 10)

numpy_data = np.column_stack((ms, mx, Vl_t))


pandaframe3 = pd.DataFrame(numpy_data, columns= ("ms", "mx", "Vl"), index = t)

print(pandaframe3)

pandaframe3.index.name = ("time in h")
layout3 = dict(title = " m = c * V : Substratemass,Biomass, Volume over time", xaxis= dict(title= "time in h"),  yaxis = dict(  title= "mass in g and volume in l"))


pandaframe3.iplot(layout = layout3 )

#print(pandaframe3)




                     ms           mx         Vl
0.000000   2.500000e+02   150.000000   5.000000
0.151515   2.606193e+02   161.330394   5.303030
0.303030   2.708972e+02   173.505662   5.606061
0.454545   2.808111e+02   186.591060   5.909091
0.606061   2.903367e+02   200.656452   6.212121
...                 ...          ...        ...
14.393939 -9.049537e-06  2650.304840  20.002439
14.545455  3.817982e-06  2650.304808  20.002439
14.696970  2.087668e-07  2650.304817  20.002439
14.848485  3.666605e-06  2650.304808  20.002439
15.000000  1.234879e-05  2650.304787  20.002439

[100 rows x 3 columns]


In [ ]:
# von matplotlib plot settings von vorher, nur für mein Gedächtnis


cs_t = result[:, 0]
cx_t = result[:, 1]
Vl_t = result[:, 2]


plt.plot(t, cs_t, "r-")
plt.plot(t, cx_t, "b-")
plt.plot(t, Vl_t, "g-")
plt.legend(["Cs", "Cx", "Vl"])
plt.show()